### Random Forest

# Discretizzazione
Andiamo a definire delle classi sul prezzo per poter usare il classificatore

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer
import matplotlib.pyplot as plt

X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
Y_train = pd.read_csv('../data/Y_train.csv')
Y_test = pd.read_csv('../data/Y_test.csv')

enc = KBinsDiscretizer(n_bins=20, encode="ordinal", strategy="quantile")
Y_train_binned = enc.fit_transform(Y_train)
Y_test_binned = enc.fit_transform(Y_test)

Y_train_binned = Y_train_binned.ravel()
Y_test_binned=Y_test_binned.ravel()

# Random Forest

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV


def RandomForestSelector():
    #Tuning della random Forest classifier
    n_trees= [27]
    #[5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,25,27,30]

    max_tree = 5
    max_acc = 0

    for t in n_trees:
        rf=RandomForestClassifier(n_estimators=27)
        rf.fit(X_train,Y_train_binned)
        acc_train = accuracy_score(y_true=Y_train_binned, y_pred=rf.predict(X_train))
        acc_test  = accuracy_score(y_true=Y_test_binned, y_pred=rf.predict(X_test))
        print('N trees {}: train accuracy: {}, test accuracy: {}'.format(t,acc_train,acc_test))

        properties={
            #min_sample_leaf piu piccolo possibile
            #max feature piu grande possibile
            #min_samples_split piu' piccolo possibile
            #max_leaf_nodes piu' grande possibile
            #max_samples piu' alto possibile
        }

        grid=GridSearchCV(rf,properties,scoring="neg_mean_squared_error",cv=8,n_jobs=-1,return_train_score=True,verbose=2, refit=True)
        grid.fit(X_train,Y_train_binned)
        acc_train = accuracy_score(y_true=Y_train_binned, y_pred=grid.predict(X_train))
        acc_test  = accuracy_score(y_true=Y_test_binned, y_pred=grid.predict(X_test))

        print('Grid: train accuracy: {}, test accuracy: {}'.format(t,acc_train,acc_test))
        print()
        print(grid.score(X_test))

        cm = ConfusionMatrixDisplay.from_estimator(
            estimator=grid,
            X=X_test, y=Y_test_binned)
        fig = cm.ax_.get_figure()
        fig.set_figwidth(12)
        fig.set_figheight(12)

RandomForestSelector()



N trees 27: train accuracy: 1.0, test accuracy: 0.26109215017064846
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Grid: train accuracy: 27, test accuracy: 0.9995733788395904



TypeError: Singleton array array(None, dtype=object) cannot be considered a valid collection.